<a href="https://colab.research.google.com/github/bennyb-bio/Group-A-Project/blob/test/covid_19_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import sqlite3
import pandas as pd
import numpy as np
import os

In [4]:
# Verzeichnis mit den CSV-Dateien
material_path = "/covid19_data"

In [5]:
# SQL-Datenbank, die im Skript 'covid_19_db' erstellt wurde 
database_file = "covid19_data.db"

In [6]:
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

In [120]:
# Mapping-Tabelle von SNOMED nach ICD 
## erstellt von ChatGPT
map_icd = pd.read_csv("map_snomed_icd.csv", sep = ";")
map_icd.head()

# Mapping-Tabelle von LOINC zu SNOMED-CT
## erstellt von ChatGPT
map_loinc = pd.read_csv("map_loinc_snomed.csv", sep = ";")
map_loinc.snomed= map_loinc.snomed.str.strip()
map_loinc.snomed = map_loinc.snomed.replace("NA", np.nan) # NAs korrekt kodieren
map_loinc.head()

,loinc,loinc_desc,snomed,snomed_desc
0,14804-9,Lactate dehydrogenase [Enzymatic activity/volu...,NaN,NaN
1,1742-6,Alanine aminotransferase [Enzymatic activity/v...,163515005,Alanine aminotransferase measurement
2,1751-7,Albumin [Mass/volume] in Serum or Plasma,104926008,Serum albumin measurement
3,17861-6,Calcium [Mass/volume] in Serum or Plasma,279000006,"Calcium measurement, serum"
4,1920-8,Aspartate aminotransferase [Enzymatic activity...,163513002,Aspartate aminotransferase measurement


# Mapping

## Tabelle conditions

Tabelle conditions enthält Diagnosen in Form von SNOMED-CT-Codes. Mapping auf ICD-Klassifikation

In [11]:
# conditions table 
# Alle distinkten SNOMED-Codes aus Tabelle conditions laden und mit Mapping-Tabelle verknüpfen
query = """
SELECT DISTINCT code as snomed, description as snomed_desc
FROM conditions
ORDER BY code desc
"""

df = pd.read_sql_query(query, conn)
df_mapped = df.merge(map_icd, how="left", on = ["snomed", "snomed_desc"])
df_mapped

,snomed,snomed_desc,icd,icd_desc
0,132281000119108,Acute deep venous thrombosis (disorder),I82.409,Acute venous embolism and thrombosis of unspec...
1,840544004,Suspected COVID-19,U07.2,"COVID-19, virus not identified"
2,840539006,COVID-19,U07.1,COVID-19
3,770349000,Sepsis caused by virus (disorder),A41.9,"Sepsis, unspecified"
4,706870000,Acute pulmonary embolism (disorder),I26.9,"Pulmonary embolism, unspecified"
5,429007001,History of cardiac arrest (situation),Z86.73,Personal history of ischemic heart disease
6,422587007,Nausea (finding),R11.0,Nausea
7,410429000,Cardiac Arrest,I46.9,"Cardiac arrest, unspecified"
8,399211009,History of myocardial infarction (situation),Z86.73,Personal history of ischemic heart disease
9,389087006,Hypoxemia (disorder),R09.02,Hypoxemia


In [12]:
print(f"Tabelle conditions:\nFür {df_mapped.icd.notna().sum()} von {len(df_mapped)} SNOMED-CT-Codes konnte ein passender ICD-Code gefunden werden")



Tabelle conditions:
Für 41 von 41 SNOMED-CT-Codes konnte ein passender ICD-Code gefunden werden


## Tabelle observations

Tabelle observations enthält Befunde in Form von LOINC-Codes. Mapping auf SNOMED-CT

In [122]:
query = """
SELECT distinct code as loinc, description 
FROM observations
ORDER BY code
"""

df = pd.read_sql_query(query, conn)
df.columns = df.columns.str.lower()
df = df.dropna(subset = ["loinc"])
#gültige LOINC-Codes (format: nnnnn-n)
df = df[df["loinc"].str.match(r"^\d{4,5}-\d$")]

df_mapped = df.merge(map_loinc, how="left", on="loinc")
df_mapped



,loinc,description,loinc_desc,snomed,snomed_desc
0,14804-9,Lactate dehydrogenase [Enzymatic activity/volu...,Lactate dehydrogenase [Enzymatic activity/volu...,NaN,NaN
1,1742-6,Alanine aminotransferase [Enzymatic activity/v...,Alanine aminotransferase [Enzymatic activity/v...,163515005,Alanine aminotransferase measurement
2,1751-7,Albumin [Mass/volume] in Serum or Plasma,Albumin [Mass/volume] in Serum or Plasma,104926008,Serum albumin measurement
3,17861-6,Calcium [Mass/volume] in Serum or Plasma,Calcium [Mass/volume] in Serum or Plasma,279000006,"Calcium measurement, serum"
4,1920-8,Aspartate aminotransferase [Enzymatic activity...,Aspartate aminotransferase [Enzymatic activity...,163513002,Aspartate aminotransferase measurement
5,1960-4,Bicarbonate [Moles/volume] in Arterial blood,Bicarbonate [Moles/volume] in Arterial blood,11557-4,Bicarbonate (HCO3-) measurement in arterial blood
6,1975-2,Bilirubin.total [Mass/volume] in Serum or Plasma,Bilirubin.total [Mass/volume] in Serum or Plasma,28773-4,"Bilirubin measurement, total (serum or plasma)"
7,1988-5,C reactive protein [Mass/volume] in Serum or P...,C reactive protein [Mass/volume] in Serum or P...,302635004,C reactive protein measurement
8,19994-3,Oxygen/Inspired gas setting [Volume Fraction] ...,Oxygen/Inspired gas setting [Volume Fraction] ...,NaN,NaN
9,2019-8,Carbon dioxide [Partial pressure] in Arterial ...,Carbon dioxide [Partial pressure] in Arterial ...,250608003,Carbon dioxide partial pressure measurement


In [126]:
print(f"Tabelle observations:\nFür {df_mapped.snomed.notna().sum()} von {len(df_mapped)} LOINC-Codes konnte ein passender SNOMED-CT-Code gefunden werden")


Tabelle observations:
Für 37 von 52 LOINC-Codes konnte ein passender SNOMED-CT-Code gefunden werden


In [ ]:
# Verbindung zur Datenbank lösen
conn.close()